<a href="https://colab.research.google.com/github/jtsuvile/AI_utbildning/blob/main/ML_python_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# **Maskininlärning med Python**
###**Mål: Bygg en klassificeringmodell för att predicera bröstcancer**

### **Kod & data:**
- URL till original-notebooken: **https://tinyurl.com/46c7ww78**
  - Ladda upp och kör i ert eget google colab-konto
- Vi kommer ladda in ett öppet dataset med data från bröstcancerpatienter. Mer information om datasettet finns [här](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic).
  - Datasetet är redan städat, transformerat och bearbetat, så vi kan använda det som det är
  - <font color="red">**OBS! Ladda aldrig upp sekretessbelagd data (t.ex. patientdata) på google colab!**</font>

### **Bra att veta:**
- Kör en specifik kod-cell genom att trycka på play-knappen i övre vänstra hörnet av cellen eller genom shift+enter
  - Kom ihåg att ordningen som du kör cellerna i spelar roll, om det finns kod i cell A som behövs i cell B så behöver du köra A före B
  - Om kod i cell B skriver över variabler i cell A, så kommer det påverka vad som händer om du kör A igen



## Steg 1: prep
Vi preppar notebooken med paket och data vi kommer behöva under övningen

In [ ]:
#Först importerar vi de paket vi behöver:

import pandas as pd # pandas är bra för att hantera tabulärdata
import matplotlib.pyplot as plt # matplotlib är för att göra snygga plottar
from sklearn import datasets # sklearn har mycket ML-relaterat, här importerar vi funktionen som ger oss datasets
from sklearn.model_selection import train_test_split # och här lite funktioner vi kommer behöva

pd.options.display.max_columns = None  # Ställer in så pandas dataframes inte kollapsar (döljer) kolumner när vi skriver ut en tabell
plt.rcParams['figure.figsize'] = [20, 10]


In [ ]:
# vi laddar in datasettet

X, y = datasets.load_breast_cancer(return_X_y=True, as_frame=True)    # Laddar X- och y-data
# X innehåller alla variabler som vi vill använda som input vid prediktion
# Y innehåller den variabel som är vår modells "target", d.v.s det modellen predicerar
y = y.apply(lambda x: datasets.load_breast_cancer().target_names[x])  # Lägger till etiketter till y-data


---
## Steg 2: Bekanta er med datasetet

Vi har laddat in data som en DataFrame, dvs en excel-liknande tabell. Paketet Pandas (alias pd) som vi laddade innan har en hel del bra funktioner som hjälper med att förstå och hantera en DataFrame.


In [ ]:
# Första frågan är alltid: hur ser datan ut?
X.head() # kommandot head() ger 5 första rader av en DataFrame

In [ ]:
# vi kan även fråga vad har vi för kolumner?
X.columns

In [ ]:
# vi kollar även på y
y.head()

### Reflektion

Vad märker du med kolumnnamn? Är det någon/några termer som upprepas ofta? Vad tror vi det handlar om?

Mer information om datasettet hittar du t.ex. [här](https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data).


### Vi fortsätter utreda data

Vi fortsätter försöka förstå vår dataset genom att utnyttja Pandas functions.

In [ ]:
# Vilka kategorier har vi i y? Hur många av varje?
y.value_counts()

In [ ]:
# hur kan vi sammanfatta variabler i X? ja, det finns en enkel funktion för det!
X.describe()

In [ ]:
# men usch det är jobbigt att kolla på siffror på det sättet. Hade det varit enklare att förstå en bild?
X.plot.box(subplots=True)
plt.show() # in order for the plot to show on the notebook, you need to always call plt.show after creating a plot

In [ ]:
# det var lite bra men inte helt och hållet. Vi fortsätter utveckla plottet för att det ska vara lite mer informativt!
# normalt hade du fortsätt genom att ändra den befintliga koden och bara kört om cellen, men för att vara pedagogiska kommer vi göra det steg för steg

# vi lägger in en arguemnt som heter "layout", den hjälper oss få plottarna lite tydligare
X.plot.box(subplots=True, legend=False, layout=(3, 10))
plt.show()
# blev det bättre? känn dig fri att ändra på antal (rader, kolumner) i layout för att hitta vad som känns rimligt för dig

In [ ]:
# vi kan även leka lite med plottyp
# alla möjliga typer finns här https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html#pandas.DataFrame.plot
# men alla kommer inte se lika snygga ut!
X.plot.density(subplots=True, layout=(3, 10), legend=False)
plt.show()

In [ ]:
 # beroende på plottyp vill vi använda lite olika argument för att det ska bli bra
X.plot.density(subplots=True, layout=(3, 10), legend=False, sharex=False,
               title=X.columns.tolist())
plt.tight_layout()
plt.show()

In [ ]:
# vi kan även kolla på variabler två och två med hjälp av korrelationsmatris
X.corr()

In [ ]:
# oj nej det blev massa siffror igen, svårt att kolla igenom dem om de är i numerisk format.
# vi gör ett plot av det istället
plt.matshow(X.corr())
plt.show()

In [ ]:
# det var bättre men det är svårt att veta vilken siffra motsvarar vilken färg
# så vi lägger in en colorbar
p1 = plt.matshow(X.corr())
plt.colorbar(p1)
plt.show()

### Reflektion

Innan vi kan gå vidare till maskinlärning, det är viktigt att vi förstår om det är någonting i datasettet som känns konstigt eller som behöver hanteras. Är variabler kategoriska eller numeriska? Om du inte kommer ihåg var det innebär med kategoriska och numeriska variabler, känn dig fri att fråga chattboten (och sen verifiera från någon annan källa). Har alla samma skala? Är det mycket outliers? Finns det några värden som korrelerar mycket starkt med varandra (r > 0.9 eller så)?

---
## Steg 3: Dela upp test och train
I maskininlärning är det otroligt viktigt att testdata inte får läcka sig in i träningsstadiet. För att undvika detta, vill vi dela upp datasettet i test och train innan vi börjar göra något vidare med det.

In [ ]:
# Dela upp dataset till träning- och testdata
# X_train och y_train är data som ska användas vid träning av modellen
# X_test och y_test är data som ska användas vid utvärdering av modellen
# att vi explicit sätter random state gör så att vi alla kommer få samma individer i test och train
# vilket gör det enklare att troubleshootta och debugga om det behövs!

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1234)

### Reflektion
Varför är det dåligt om testdata läcker in i träningsdata? Om detta inte känns självklart, fråga gärna chattboten (och verifiera svar från någon annan källa)!

----
## Steg 4: Standardisering av X-data

En problem vi kan ha upptäckt är att variabler har väldigt olika storleksgrad. Det klarar inte alla maskininlärningsalgoritmer av. Därför vill vi standardisera våra numeriska variabler innan vi tränar modellen.

Standardisering gör det dessutom lättare att tolka samband och jämföra variabler mot varandra. Standardiserad data har följande egenskaper:
- medelvärde=0
- standardavvikelse=1

I detta fall använder ett funktion som heter `StandardScaler()` för att standardisera vår numeriska X-data (vi standardiserar inte y-datan eftersom den är kategorisk). Det finns även andra typer av Scaler funktioner, t.ex. `MinMaxScaler()` för olika användningsfall.


In [ ]:
from sklearn.preprocessing import StandardScaler

# Definiera vår scaler
scaler = StandardScaler()

# Vi tränar scalern enbart på träningsdata
X_train_scaled = scaler.fit_transform(X_train)

# Vi tillämpar scalern även på testdata
X_test_scaled = scaler.transform(X_test)


In [ ]:
# vi kan kolla visuellt vad har hänt med våra distributioner efter scalern
# kolla noga på x-axis!
pd.DataFrame(X_train_scaled).plot.density(subplots=True, layout=(3, 10), legend=False, sharex=False,
               title=X.columns.tolist())
plt.tight_layout()
plt.show()

---
# Steg 5: Träning och prediktion

Vi ska använda den skalade träningsdatan för att träna en modell, och sedan den skalade testdatan för att ta fram prediktioner.

Slutligen så ska vi räkna ut en confusion matrix där vi kan se hur många av de predicerade värdena för respektive utfall som faktiskt var korrekta enligt "facit".


In [ ]:
# vi börjar med att importera lite fler funktioner som vi behöver
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
# Välj ett modell
model = KNeighborsClassifier()

# Träna modell (det här är den biten som alla hajpar!)
model.fit(X_train_scaled, y_train)

# Nu tar vi ett dataset som modellen inte har set förut (test settet)
# och ber den prediktera klass på de exempel som finns med på det
y_pred = model.predict(X_test_scaled)

In [ ]:
# Vi gör ett första koll på hur bra det blev genom att titta på confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm = pd.DataFrame(cm, columns=["predicted_benign", "predicted_maglignant"], index=["actually_benign", "actually_malignant"])
display(cm)

### Uppgift
Försök byta ut andra modeller till raden som börjar med `model = `. Blir det bättre eller sämre resultat? Vilken modell ger bäst resultat?


### Reflektion
Det finns två olika typer av fel man kan få i confusion matrixen ovan: false positives (klassificeras som positiv, facit negativ) och false negatives (klassificeras som negativ, facit positiv). Matematiskt räknas båda två oftast att vara "lika dåliga", men är det så?

Kom ihåg att detta är en klinisk uppgift i grunden: prediktera vilka patienter är friska och vilka är sjuka. Om man tänker på den kliniska tillämpningen, är det skillnad mellan att ha false positives (man är frisk men klassificeras som sjuk) och false negatives (man är sjuk men klassificeras som frisk)? Tycker ni att någon av de två är allvarligare än den andra?

---
## Steg 5: Utvärdera modellen

Vi kan utvärdera modellen utifrån ett antal olika mått. Vi har redan kollat på confusion matrix, men det kan vara svårt att veta om något blev bättre eller sämre. Därför finns det olika prestandamått som med en siffra ger oss en hum om hur "bra" modellen blev.

Vi kommer testa den mest vanliga, som heter accuracy. Det är bra att veta att även om accuracy används mycket, det vara missvisande när datan i y är väldigt obalanserad (t.ex. om nästan alla fall har samma utfall). Om datan är mycket obalanserat, använder man oftast någon annan prestandamått eller fler prestandamått tillsammans.


In [ ]:
# vi använder befintlig funktion för att räkna fram accuracyn
accuracy_score(y_test, y_pred)

In [ ]:
# accuracy är en ganska enkel mått att även räkna fram själv
# vi räknar antal prediktioner som blev helt rätt
number_of_correct_predictions = sum(y_test == y_pred)
# antal prediktioner som vi gjorde totalt
number_of_predictions = len(y_pred)
# och räknar andel rätt av allt
accuracy = number_of_correct_predictions / number_of_predictions
# om vi bara skriver namn på variabel så kommer den skrivas ut efter cellen!
accuracy

---
# Steg 7: Fri lek

Har vi tid kvar och ni har hunnit hela vägen hit får ni gärna gå till den biten av kod som kändes roligast och se om ni kan ändra på någonting själva. Några idéer:

1. Kan ni skapa mer informativa plottar om data eller snygga till de som vi har gjort tillsammans? [Kolla tips t.ex. här](https://matplotlib.org/stable/gallery/index.html)
2. Kan ni hitta fler maskininlärningsmodeller som ni kan testa träna modellen på? [Kolla modeller som finns inom Scikit-learn här](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html)
3. Kan ni ta fram någon annan prestandamått för resultatet? [Här finns alla metriker för Scikit-learn](https://scikit-learn.org/stable/modules/model_evaluation.html#)
4. Kan ni hitta sätt att avrunda resultatet av accuracy beräkning till 2 decimaler? Googla!

Kom ihåg att du kan behöva importera nya funktioner för att lösa uppgifterna!

### Reflektion

Hur kändes det?  

Har du tips om hur man kan förbättra uppgifterna/notebooken för framtida omgångar får du gärna höra av dig till juulia.suvilehto@vgregion.se .